# **Homework**: Data talks club data engineering zoomcamp Data loading workshop


# Question 1

In [ ]:
def square_root_generator(limit):
   n = 1
   while n <= limit:
       yield n ** 0.5
       n += 1


# Example usage:
limit = 5
generator = square_root_generator(limit)


res = 0
for sqrt_value in generator:
   res += sqrt_value
print(f"The sum of the outputs of the generator for limit = 5 is {res}")

The sum of the outputs of the generator for limit = 5 is 8.382332347441762


# Question 2

In [2]:
def square_root_generator(limit):
   n = 1
   while n <= limit:
       yield n ** 0.5
       n += 1


# Example usage:
limit = 13
generator = square_root_generator(limit)


for sqrt_value in generator:
   print(sqrt_value)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989


# 2. Append a generator to a table with existing data

In [5]:
%%capture
!pip install dlt[duckdb] # Install dlt with all the necessary DuckDB dependencies

In [6]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [7]:
%%capture
!pip install dlt

Creating the table "people" using the first generator  

In [8]:
import dlt

generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')


# we can load any generator to a table at the pipeline destnation as follows:
info = generators_pipeline.run(people_1(),
										table_name="people",
										write_disposition="replace")

# the outcome metadata is returned by the load and we can inspect it by printing it.
print(info)


Pipeline dlt_colab_kernel_launcher load step completed in 0.38 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707585024.5613859 is LOADED and contains no failed jobs


Appending to the table people the second generator

In [9]:
# we can load any generator to a table at the pipeline destnation as follows:
info = generators_pipeline.run(people_2(),
										table_name="people",
										write_disposition="append")

# the outcome metadata is returned by the load and we can inspect it by printing it.
print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.34 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707585025.3975928 is LOADED and contains no failed jobs


# Question 3
Calculate the sum of all ages of people.

In [10]:
import duckdb

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

people = conn.sql("SELECT SUM(age) FROM people").df()
display(people)


Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘

,sum(age)
0,353.0


# 3. Merge a generator

Re-use the generators from Exercise 2.

Creating a new table called people2 using the first generator using the "ID" as a primary key.

In [12]:
import dlt

generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')


# we can load any generator to a table at the pipeline destnation as follows:
info = generators_pipeline.run(people_1(),
										table_name="people2",
										write_disposition="replace",
                    primary_key="ID")

# the outcome metadata is returned by the load and we can inspect it by printing it.
print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.25 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707585335.09256 is LOADED and contains no failed jobs


Merging data from the second generator into the table people2 using the primary key parameter.

In [13]:
# we can load any generator to a table at the pipeline destnation as follows:
info = generators_pipeline.run(people_2(),
										table_name="people2",
										write_disposition="merge",
                    primary_key="ID")

# the outcome metadata is returned by the load and we can inspect it by printing it.
print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.42 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707585339.1579382 is LOADED and contains no failed jobs


# Question 4

In [15]:
print('Loaded tables: ')
display(conn.sql("show tables"))

people = conn.sql("SELECT SUM(age) FROM people2").df()
display(people)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
│ people2             │
└─────────────────────┘

,sum(age)
0,266.0
